In [13]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

finbert = BertForSequenceClassification.from_pretrained('ProsusAI/finbert')
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')

sentiment_pipeline = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

Device set to use cpu


In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("translated_articles_index.csv", sep='|', encoding='UTF-8', dtype='str')

In [16]:
df

,full_text,translated_content,date,url,title,content
0,"Új kör szekerét tolja meg a kormány, „ingyenpé...","The government is launching a new initiative, ...",2024/12/31,https://index.hu/gazdasag/2024/12/31/demjan-sa...,"Új kör szekerét tolja meg a kormány, „ingyenpé...",A hazai kis- és középvállalkozások hóna alá ny...
1,Financial Times: Magyarország több mint egymil...,Financial Times: Hungary Could Lose Over a Bil...,2024/12/31,https://index.hu/gazdasag/2024/12/31/magyarors...,Financial Times: Magyarország több mint egymil...,"A brit gazdasági szaklap arról írt, hogy Magya..."
2,Orbán Balázs: A vendégmunkások nem maradnak az...,**Orbán Balázs: Guest Workers Will Not Stay in...,2024/12/31,https://index.hu/gazdasag/2024/12/31/orban-bal...,Orbán Balázs: A vendégmunkások nem maradnak az...,"Magyarország gazdasági kilátásairól, az eddig ..."
3,Túladóztat a kormány a szakértő szerint – ez v...,"According to an expert, the government is over...",2024/12/31,https://index.hu/gazdasag/2024/12/31/uzemanyag...,Túladóztat a kormány a szakértő szerint – ez v...,Szerdától emelkedik a benzin és a gázolaj nagy...
4,Januártól záporoznak a fontos pénzügyi változá...,"From January, important financial changes will...",2024/12/31,https://index.hu/gazdasag/2024/12/31/penzugyek...,Januártól záporoznak a fontos pénzügyi változá...,"Felsorolni is nehéz, mennyi változás történik ..."
...,...,...,...,...,...,...
25813,Fellebbez a Paks II.-ből kizárt orosz cég. Kif...,The Russian company excluded from Paks II. is ...,2018/01/02,https://index.hu/gazdasag/2018/01/02/fellebbez...,Fellebbez a Paks II.-ből kizárt orosz cég,Kifogást nyújtott be az orosz Szilovije Masini...
25814,5 milliárd forintnyi EU-s pénzt nyert az NGM c...,The National Economy Ministry (NGM) has won 5 ...,2018/01/02,https://index.hu/gazdasag/2018/01/02/5_milliar...,5 milliárd forintnyi EU-s pénzt nyert az NGM c...,Újra nekifuthat a kormány a közszféra leépítés...
25815,Saját futárszolgáltatot indít az Edigital. Saj...,Edigital is launching its own courier service....,2018/01/02,https://index.hu/gazdasag/2018/01/02/sajat_fut...,Saját futárszolgáltatot indít az Edigital,Saját futárszolgálat indítását és első külföld...
25816,Igenis olcsók a budapesti lakások. Frissebb (1...,"Yes, apartments in Budapest are cheap.\n\n**Is...",2018/01/01,https://index.hu/gazdasag/2018/01/01/igenis_ol...,Igenis olcsók a budapesti lakások,Frissebb (1) Vége az őrületnek a magyar lakásp...


In [17]:
def split_text_equal_parts(text, max_tokens=500):
    """Splits text into equal-length chunks based on its total tokenized length"""
    token_ids = tokenizer.encode(text, add_special_tokens=False)  
    num_tokens = len(token_ids)

    if num_tokens <= max_tokens:
        return [tokenizer.decode(token_ids)]  # Return single chunk

    num_chunks = int(np.ceil(num_tokens / max_tokens))  # Number of equal chunks
    chunk_size = int(np.floor(num_tokens / num_chunks))  # Compute chunk size

    chunks = [tokenizer.decode(token_ids[i:i+chunk_size]) for i in range(0, num_tokens, chunk_size)]
    return chunks

In [18]:
def get_sentiment_score(text):
    """Splits text into equal parts, gets sentiment probability scores, and averages"""
    chunks = split_text_equal_parts(text)
    scores = []

    for chunk in chunks:
        results = sentiment_pipeline(chunk, return_all_scores=True)[0]  # Get all probabilities
        
        # Extract probabilities for each sentiment
        neg_prob = next(item['score'] for item in results if item['label'] == 'negative')
        neu_prob = next(item['score'] for item in results if item['label'] == 'neutral')
        pos_prob = next(item['score'] for item in results if item['label'] == 'positive')

        # Compute sentiment score: weighted sum of probabilities
        sentiment_score = pos_prob - neg_prob  # Range: -1 (fully negative) to +1 (fully positive)
        scores.append(sentiment_score)

    return np.mean(scores) if scores else 0  # Average sentiment score across chunks

# Apply to DataFrame
df['sentiment_score'] = df['translated_content'].apply(get_sentiment_score)

df_final = df.loc[:,['translated_content', 'date', 'sentiment_score']]

df_final.to_csv("sentiment_scores_index.csv", sep="|", encoding='utf-8', index=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (1144 > 512). Running this sequence through the model will result in indexing errors
C:\Users\tothk\anaconda3\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [6]:
df_final

,translated_content,date,sentiment_score
0,Construction workers urgently need to take out...,2025/01/07,-0.312391
1,An employee of the Békéscsaba school district ...,2025/01/07,-0.788441
2,OTP and Lőrinc Mészáros's bank received more t...,2025/01/07,-0.748979
3,Márton Nagy flew on from his Parisian accommod...,2025/01/07,-0.008498
4,"If you received a gas bill like this, don't ig...",2025/01/07,-0.123949
...,...,...,...
15707,"VOSZ: No debate, layoffs are necessary in the ...",2017/12/08,-0.731472
15708,There might be problems in the construction in...,2017/12/08,-0.842442
15709,The alarm bells are ringing: the Hungarian eco...,2017/12/08,0.004161
15710,## Private Pension Assets: The 2826 Billion Fo...,2017/12/08,-0.009108
